# FizzBuzz問題をニューラルネットワークで解いてみる

## TL;DR

FizzBuzz問題をニューラルネットワークで解いてみます。

## ラベルデータの作成

In [1]:
import pandas as pd

results = []

for i in range(1, 10000 + 1):
    if i % 3 == 0 and i % 5 == 0:
        results.append((i, 'FizzBuzz'))
    elif i % 3 == 0:
        results.append((i, 'Fizz'))
    elif i % 5 == 0:
        results.append((i, 'Buzz'))
    else:
        results.append((i, 'Number'))

data_df = pd.DataFrame(results, columns=['Number', 'Results'])
display(data_df.head(15))

,Number,Results
0,1,Number
1,2,Number
2,3,Fizz
3,4,Number
4,5,Buzz
5,6,Fizz
6,7,Number
7,8,Number
8,9,Fizz
9,10,Buzz


## 前処理

In [2]:
feature_title = 'Number'
label_title = 'Results'

printable_labels = {k: i for i, k in enumerate(data_df[label_title].unique())}
class_count = len(printable_labels)

display(data_df.head(15))
display(data_df.describe())
display(printable_labels)

,Number,Results
0,1,Number
1,2,Number
2,3,Fizz
3,4,Number
4,5,Buzz
5,6,Fizz
6,7,Number
7,8,Number
8,9,Fizz
9,10,Buzz


,Number
count,10000.00000
mean,5000.50000
std,2886.89568
min,1.00000
25%,2500.75000
50%,5000.50000
75%,7500.25000
max,10000.00000


{'Number': 0, 'Fizz': 1, 'Buzz': 2, 'FizzBuzz': 3}

In [3]:
from keras import utils

labels = utils.np_utils.to_categorical([printable_labels[label] for label in data_df[label_title]], num_classes=class_count)

display(labels.shape)
display(labels)

C:\Users\hidek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(10000, 4)

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [4]:
import numpy as np

digits_count = 5

didgits_map = utils.np_utils.to_categorical(range(10), num_classes=10)
features = np.zeros((len(data_df), 5, 10), dtype = np.int32)

for i, number in enumerate(data_df[feature_title]):
    for t, digit in enumerate(str(number).zfill(digits_count)[:]):
        features[i, t] = didgits_map[int(digit)]

print(features.shape)
print(features)

(10000, 5, 10)
[[[1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 1 0]]

 [[1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]]

 [[0 1 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]]]


## データの分割

In [5]:
from sklearn.model_selection import train_test_split

idx_features = range(len(data_df[feature_title]))
idx_labels = range(len(data_df[label_title]))
tmp_data = train_test_split(idx_features, idx_labels, train_size = 0.9, test_size = 0.1)

train_features = np.array([features[i] for i in tmp_data[0]])
valid_features = np.array([features[i] for i in tmp_data[1]])
train_labels = np.array([labels[i] for i in tmp_data[2]])
valid_labels = np.array([labels[i] for i in tmp_data[3]])

print(train_features.shape)
print(valid_features.shape)
print(train_labels.shape)
print(valid_labels.shape)

(9000, 5, 10)
(1000, 5, 10)
(9000, 4)
(1000, 4)


## ネットワークの作成

In [6]:
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, RepeatVector, Permute, Flatten, SimpleRNN
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras import Input, Model

model = Sequential()
model.add(Flatten(input_shape=(features[0].shape)))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(class_count, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              104448    
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 8196      
Total params: 4,308,996
Trainable params: 4,308,996
Non-trainable params: 0
_________________________________________________________________


## 学習

In [7]:
from keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint, TensorBoard

model_filename = 'models/fizzbuzzz-model.h5'

history = model.fit(train_features,
          train_labels,
          epochs = 300,
          validation_split = 0.1,
          batch_size = 256,
          callbacks = [
              TensorBoard(log_dir = 'logs'),
              EarlyStopping(patience=5, monitor='val_mean_absolute_error'),
              ModelCheckpoint(model_filename, monitor='val_mean_absolute_error', save_best_only=True)
          ])

Train on 8100 samples, validate on 900 samples
Epoch 1/300
8100/8100 [==============================] - 1s 137us/step - loss: 0.7933 - mean_absolute_error: 0.2498 - val_loss: 0.6256 - val_mean_absolute_error: 0.2134
Epoch 2/300
8100/8100 [==============================] - 0s 41us/step - loss: 0.6548 - mean_absolute_error: 0.2246 - val_loss: 0.6258 - val_mean_absolute_error: 0.2135
Epoch 3/300
8100/8100 [==============================] - 0s 39us/step - loss: 0.6422 - mean_absolute_error: 0.2232 - val_loss: 0.6292 - val_mean_absolute_error: 0.2229
Epoch 4/300
8100/8100 [==============================] - 0s 40us/step - loss: 0.6318 - mean_absolute_error: 0.2208 - val_loss: 0.6136 - val_mean_absolute_error: 0.2144
Epoch 5/300
8100/8100 [==============================] - 0s 41us/step - loss: 0.5816 - mean_absolute_error: 0.2072 - val_loss: 0.5271 - val_mean_absolute_error: 0.1884
Epoch 6/300
8100/8100 [==============================] - 0s 39us/step - loss: 0.3785 - mean_absolute_error: 0.14

8100/8100 [==============================] - 0s 39us/step - loss: 2.2934e-05 - mean_absolute_error: 1.1471e-05 - val_loss: 1.4647e-04 - val_mean_absolute_error: 7.3019e-05
Epoch 47/300
8100/8100 [==============================] - 0s 39us/step - loss: 2.1527e-05 - mean_absolute_error: 1.0767e-05 - val_loss: 1.3988e-04 - val_mean_absolute_error: 6.9743e-05
Epoch 48/300
8100/8100 [==============================] - 0s 39us/step - loss: 2.0257e-05 - mean_absolute_error: 1.0133e-05 - val_loss: 1.3043e-04 - val_mean_absolute_error: 6.5047e-05
Epoch 49/300
8100/8100 [==============================] - 0s 40us/step - loss: 1.9082e-05 - mean_absolute_error: 9.5450e-06 - val_loss: 1.2418e-04 - val_mean_absolute_error: 6.1939e-05
Epoch 50/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.7998e-05 - mean_absolute_error: 9.0028e-06 - val_loss: 1.1753e-04 - val_mean_absolute_error: 5.8628e-05
Epoch 51/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.6998e-0

8100/8100 [==============================] - 0s 39us/step - loss: 3.4221e-06 - mean_absolute_error: 1.7122e-06 - val_loss: 3.0667e-05 - val_mean_absolute_error: 1.5324e-05
Epoch 91/300
8100/8100 [==============================] - 0s 39us/step - loss: 3.3174e-06 - mean_absolute_error: 1.6596e-06 - val_loss: 3.0244e-05 - val_mean_absolute_error: 1.5111e-05
Epoch 92/300
8100/8100 [==============================] - 0s 40us/step - loss: 3.2160e-06 - mean_absolute_error: 1.6090e-06 - val_loss: 2.9058e-05 - val_mean_absolute_error: 1.4519e-05
Epoch 93/300
8100/8100 [==============================] - 0s 39us/step - loss: 3.1182e-06 - mean_absolute_error: 1.5598e-06 - val_loss: 2.7994e-05 - val_mean_absolute_error: 1.3988e-05
Epoch 94/300
8100/8100 [==============================] - 0s 40us/step - loss: 3.0240e-06 - mean_absolute_error: 1.5132e-06 - val_loss: 2.7842e-05 - val_mean_absolute_error: 1.3912e-05
Epoch 95/300
8100/8100 [==============================] - 0s 39us/step - loss: 2.9343e-0

8100/8100 [==============================] - 0s 39us/step - loss: 1.0750e-06 - mean_absolute_error: 5.3159e-07 - val_loss: 1.1877e-05 - val_mean_absolute_error: 5.9332e-06
Epoch 135/300
8100/8100 [==============================] - 0s 40us/step - loss: 1.0528e-06 - mean_absolute_error: 5.2008e-07 - val_loss: 1.1681e-05 - val_mean_absolute_error: 5.8348e-06
Epoch 136/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.0296e-06 - mean_absolute_error: 5.0838e-07 - val_loss: 1.1343e-05 - val_mean_absolute_error: 5.6662e-06
Epoch 137/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.0075e-06 - mean_absolute_error: 4.9738e-07 - val_loss: 1.1257e-05 - val_mean_absolute_error: 5.6227e-06
Epoch 138/300
8100/8100 [==============================] - 0s 40us/step - loss: 9.8470e-07 - mean_absolute_error: 4.8580e-07 - val_loss: 1.0985e-05 - val_mean_absolute_error: 5.4865e-06
Epoch 139/300
8100/8100 [==============================] - 0s 39us/step - loss: 9.64

8100/8100 [==============================] - 0s 39us/step - loss: 4.5768e-07 - mean_absolute_error: 2.1401e-07 - val_loss: 5.6668e-06 - val_mean_absolute_error: 2.8227e-06
Epoch 179/300
8100/8100 [==============================] - 0s 40us/step - loss: 4.5002e-07 - mean_absolute_error: 2.0989e-07 - val_loss: 5.6616e-06 - val_mean_absolute_error: 2.8193e-06
Epoch 180/300
8100/8100 [==============================] - 0s 40us/step - loss: 4.4309e-07 - mean_absolute_error: 2.0618e-07 - val_loss: 5.4812e-06 - val_mean_absolute_error: 2.7290e-06
Epoch 181/300
8100/8100 [==============================] - 0s 40us/step - loss: 4.3576e-07 - mean_absolute_error: 2.0237e-07 - val_loss: 5.4318e-06 - val_mean_absolute_error: 2.7040e-06
Epoch 182/300
8100/8100 [==============================] - 0s 40us/step - loss: 4.2890e-07 - mean_absolute_error: 1.9867e-07 - val_loss: 5.3586e-06 - val_mean_absolute_error: 2.6671e-06
Epoch 183/300
8100/8100 [==============================] - 0s 39us/step - loss: 4.21

8100/8100 [==============================] - 0s 40us/step - loss: 2.4693e-07 - mean_absolute_error: 9.8814e-08 - val_loss: 3.0773e-06 - val_mean_absolute_error: 1.5205e-06
Epoch 223/300
8100/8100 [==============================] - 0s 40us/step - loss: 2.4417e-07 - mean_absolute_error: 9.7254e-08 - val_loss: 3.0073e-06 - val_mean_absolute_error: 1.4854e-06
Epoch 224/300
8100/8100 [==============================] - 0s 40us/step - loss: 2.4153e-07 - mean_absolute_error: 9.5709e-08 - val_loss: 2.9657e-06 - val_mean_absolute_error: 1.4641e-06
Epoch 225/300
8100/8100 [==============================] - 0s 40us/step - loss: 2.3867e-07 - mean_absolute_error: 9.4190e-08 - val_loss: 2.9465e-06 - val_mean_absolute_error: 1.4544e-06
Epoch 226/300
8100/8100 [==============================] - 0s 40us/step - loss: 2.3623e-07 - mean_absolute_error: 9.2710e-08 - val_loss: 2.9269e-06 - val_mean_absolute_error: 1.4444e-06
Epoch 227/300
8100/8100 [==============================] - 0s 40us/step - loss: 2.33

8100/8100 [==============================] - 0s 40us/step - loss: 1.6711e-07 - mean_absolute_error: 5.0280e-08 - val_loss: 1.7898e-06 - val_mean_absolute_error: 8.7049e-07
Epoch 267/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.6612e-07 - mean_absolute_error: 4.9575e-08 - val_loss: 1.7732e-06 - val_mean_absolute_error: 8.6200e-07
Epoch 268/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.6491e-07 - mean_absolute_error: 4.8891e-08 - val_loss: 1.7486e-06 - val_mean_absolute_error: 8.5002e-07
Epoch 269/300
8100/8100 [==============================] - 0s 40us/step - loss: 1.6383e-07 - mean_absolute_error: 4.8205e-08 - val_loss: 1.7237e-06 - val_mean_absolute_error: 8.3687e-07
Epoch 270/300
8100/8100 [==============================] - 0s 39us/step - loss: 1.6287e-07 - mean_absolute_error: 4.7532e-08 - val_loss: 1.7025e-06 - val_mean_absolute_error: 8.2596e-07
Epoch 271/300
8100/8100 [==============================] - 0s 40us/step - loss: 1.61

## 検証

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

predicted_valid_labels = model.predict(valid_features).argmax(axis=1)
numeric_valid_labels = np.argmax(valid_labels, axis=1)
print(classification_report(numeric_valid_labels, predicted_valid_labels, target_names=printable_labels))

             precision    recall  f1-score   support

     Number       1.00      1.00      1.00       551
       Fizz       1.00      1.00      1.00       270
       Buzz       1.00      1.00      1.00       119
   FizzBuzz       1.00      1.00      1.00        60

avg / total       1.00      1.00      1.00      1000

